# Lab 4: Search

In the second lecture, you learned about different types of agents, and in the previous lab, you played around with simple reflex agents and model-based reflex agents. In the third lecture and in this lab, we discuss goal-based agents.

If you have a problem where the state space of the problem is well-defined, AND the space of actions that can be taken at each state is well-defined, AND which states correspond to achieving the agent's goal are well-defined, then one simple and flexible method of getting the agent to achieve its goals is through a somewhat "brute force" method, which is a search through the state space until a goal state is reached. Having multiple possible actions to take at each state creates a branching structure resembling a tree, where each node *n* at level *l* of the tree represents a state reachable after taking *l* actions. Depending on which action you take at each step, you can end up in a different world (state), and what search methods do is explore these possibilities in order to decide which action to take now.

Search methods all involve enumerating the possible actions at a particular state (expanding a node), and choosing which node to expand next. You can use search algorithms that stop once they find any viable path to a goal, or you can use others that take into account *path cost* and therefore seek to find the *best* (lowest cost path) to a goal.

Depending on the search algorithm, you can spend more or less time exploring this tree, expanding many nodes, or making educated guesses about which nodes are most likely to get us closer to a goal state.

### Comparison of graph search algorithms

These <a href="https://cs.stanford.edu/people/abisee/gs.pdf">graph search slides from Stanford</a> are a good overview of some of the graph search algorithms that are out there and how they work.

I recommend working through the slides, but if you want to just go directly to the interactive visualizations, here are some direct links:

- <a href="https://cs.stanford.edu/people/abisee/tutorial/bfs.html">Breadth-first search (BFS)</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/dfs.html">Depth-first search (DFS)</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/bfsdfs.html">BFS vs. DFS comparison</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/greedy.html">Greedy best-first search</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/astar.html">A* search</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/dijkstra.html">Dijkstra's algorithm (for finding min cost path)</a>
- <a href="https://cs.stanford.edu/people/abisee/tutorial/customize.html">Dijkstra vs. weighted A* (and other comparisons)</a>

### Nonogram solver

As illustrated above, search algorithms can be used to find a path to a geographic goal. But as discussed in class, search algorithms can be used to explore more abstract spaces of possibilities, like exploring many options in order to solve puzzles. Let's use graph search in order to solve some <a href="https://en.wikipedia.org/wiki/Nonogram">nonogram puzzles</a>.

Let's get started. We'll be using search_minimal.py, which features some helpful structure to define problems, actions, how actions update state, and goal states. See the source of Problem and Node below.

In [1]:
from search_minimal import *
from notebook import psource
import numpy as np

psource(Problem) # Abstract: extend this to define a new problem.
psource(Node)    # Already finished; don't need to subclass this.

First, we have to decide on how to represent nonogram-type problems. Let's represent them as r x c grids with a list of numbers for each of the r rows and c columns. I've extended Problem to implement this Nonogram:

In [2]:
class Mark(object):
    FILLED = 1
    UNKNOWN = 0
    EMPTY = -1
        
class NonogramPuzzle(Problem):
    """Represents nonogram-type problems."""

    def __init__(self, nRows, nCols, specification, initial=None):
        """Stores size of puzzle and specification. Specification is two lists of lists:
        for each row, a list of numbers, and for each column, a list of numbers."""
        self.rows = nRows
        self.cols = nCols
        self.specification = specification
        if initial != None:
            self.initial = initial
        else:
            self.initial = np.full((nRows, nCols), Mark.UNKNOWN).tobytes()

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. For simplicity, states are string versions of numpy arrays
        as numpy will complain if you try to compare numpy arrays for equality."""
        # Because states are binary strings, we need to recover and reshape
        # them into a normal numpy array, so that's what this line is doing.
        statenp = np.frombuffer(state, dtype=int).reshape(self.rows, self.cols)
        actions = []
        for r in range(0, self.rows):
            for c in range(0, self.cols):
                if statenp[r,c] == Mark.UNKNOWN:
                    filled = np.copy(statenp)
                    filled[r,c] = Mark.FILLED
                    actions.append(filled)
                    
                    empty = np.copy(statenp)
                    empty[r,c] = Mark.EMPTY
                    actions.append(empty)
        return actions

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. This is just the action itself, 
        converted to string form."""
        return action.tobytes()
    
    def line_check(self, spec, line):
        """Helper function for the goal test. May be buggy."""
        s = 0
        pattern_started = False
        count = 0
        for l in range(line.size):
            if line[l] == Mark.UNKNOWN:
                return False
            elif line[l] == Mark.EMPTY:
                if pattern_started:
                    if count == 0:
                        pattern_started = False
                    else:
                        return False
                else: # No pattern has started yet; keep skipping spaces
                    continue
            else: # Filled mark
                if pattern_started == False:
                    if s >= len(spec):
                        return False
                    pattern_started = True
                    count = spec[s] - 1
                    s += 1
                else:
                    count -= 1
        if count == 0 and s == len(spec):
            return True
                    

    def goal_test(self, state):
        """Return True if the state is a goal (valid solution to the puzzle).
        This is fairly involved."""
        statenp = np.frombuffer(state, dtype=int).reshape(self.rows, self.cols)
        # First check if any squares are empty
        for r in range(self.rows):
            for c in range(self.cols):
                if statenp[r,c] == Mark.UNKNOWN:
                    return False
                
        # For each row and column, need to check against the specification
        for r in range(self.rows):
            if self.line_check(self.specification[0][r], statenp[r,:]):
                continue
            else:
                return False
            
        for c in range(self.cols):
            if self.line_check(self.specification[1][c], statenp[:,c]):
                continue
            else:
                return False
            
        return True


def nonogram_print(puzzle, state):
    """Helper function for printing states. Requires the puzzle in order to
    figure out what dimensions the state is supposed to be printed with."""
    statenp = np.frombuffer(state, dtype=int).reshape(puzzle.rows, puzzle.cols)
    prt_str = ""
    for r in range(statenp.shape[0]):
        for c in range(statenp.shape[1]):
            if statenp[r,c] == Mark.UNKNOWN:
                prt_str += "?"
            elif statenp[r,c] == Mark.FILLED:
                prt_str += "@"
            else:
                prt_str += " "
        prt_str += "\n"
    print(prt_str)


# Here's an example 3x3 nonogram
small_nonogram = NonogramPuzzle(3, 3, ([[1,1],[3],[1]], 
                                         [[2],[2],[2]]))

nonogram_print(small_nonogram, small_nonogram.initial)

???
???
???



### Exercise: Implement depth-first search

Now that we have the above problem definition, implement depth-first search.

In [3]:
def depth_first_search(problem):
    """Returns a node containing a valid goal state if one is found, and None otherwise."""
    
    # Start at the problem's initial state
    node = Node(problem.initial)
    
    # First, check if the current node's state (node.state) is a goal state, and (if so) return it
    # FILL ME IN!
    
    
    # We then create two structures: a frontier containing nodes we would like to explore, 
    # and a set that keeps track of all states we have already visited.
    frontier = deque([node])
    explored = set()
    
    while frontier:
        # Depth first search, so we take the most recently added item (pop())
        node = frontier.pop()

        # This adds the node to the explored set in a way that allows the set to quickly check
        # if an item has already been added (by converting np array into a string)
        explored.add(node.state)
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                # As long as the child has not already been visited and has not been added to the
                # frontier set... what should we do?
                # FILL ME IN!
                
    # FILL ME IN! What to do if we exit the above while loop?


solution = depth_first_search(small_nonogram)

if solution == None:
    print("No solution found!")
else:
    nonogram_print(small_nonogram, solution.state)

IndentationError: expected an indented block (Temp/ipykernel_3300/135039770.py, line 33)

### Exercise: Implement breadth-first search

We can easily convert the above algorithm to breadth-first search by changing the frontier from a stack into a queue. This is a one-line change at the top of the while loop. But... what happens?

### Exercise: Implement greedy search

We want to try to solve these puzzles faster. (Of course, you can implement an algorithm that reasons through the puzzle and iteratively figures out what certain squares have to be. But we're not going to do that here.) Let's instead use a heuristic function to help us decide which paths in the tree to prioritize.

What would be a helpful heuristic to use here?

In [ ]:
def best_first_search(problem, f=None):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize."""
    # This line allows caching of f values as they are computed.
    f = memoize(f or problem.h, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None



class NonogramPuzzleH(NonogramPuzzle):
    def __init__(self, nRows, nCols, specification, initial=None):
        super().__init__(nRows, nCols, specification, initial)
        
    def h(self, node):
        # IMPLEMENT ME!
        



small_nonogram = NonogramPuzzleH(3, 3, ([[1,1],[3],[1]], 
                                         [[2],[2],[2]]))
solution = best_first_search(small_nonogram)

if solution == None:
    print("No solution found!")
else:
    nonogram_print(small_nonogram, solution.state)